In [1]:
#导入相关库
try:
    import xml.etree.cElementTree as ET  #解析xml的c语言版的模块
except ImportError:
    import xml.etree.ElementTree as ET
import cv2
import numpy as np
import math
from scipy.spatial import Voronoi
from scipy.spatial import Delaunay
from scipy.spatial import voronoi_plot_2d
import matplotlib.pyplot as plt

In [2]:
#解析图片，获取图片标注位置
def GetAnnotBoxLoc(AnotPath):#AnotPath VOC标注文件路径
    tree = ET.ElementTree(file=AnotPath)  #打开文件，解析成一棵树型结构
    root = tree.getroot()#获取树型结构的根
    ObjectSet=root.findall('object')#找到文件中所有含有object关键字的地方，这些地方含有标注目标
    ObjBndBoxSet={} #以目标类别为关键字，目标框为值组成的字典结构
    for Object in ObjectSet:
        ObjName=Object.find('name').text
        BndBox=Object.find('bndbox')
        x1 = int(BndBox.find('xmin').text)#-1 #-1是因为程序是按0作为起始位置的
        y1 = int(BndBox.find('ymin').text)#-1
        x2 = int(BndBox.find('xmax').text)#-1
        y2 = int(BndBox.find('ymax').text)#-1
        BndBoxLoc=[x1,y1,x2,y2]
        if ObjName in ObjBndBoxSet:
            ObjBndBoxSet[ObjName].append(BndBoxLoc)#如果字典结构中含有这个类别了，那么这个目标框要追加到其值的末尾
        else:
            ObjBndBoxSet[ObjName]=[BndBoxLoc]#如果字典结构中没有这个类别，那么这个目标框就直接赋值给其值吧
    return ObjBndBoxSet
crossing = GetAnnotBoxLoc('C:/Users/lurchy/Desktop/EasyDL/8230-4502-0.5.xml')

In [3]:
#循环计算各标注位置的垂直梯度，将超过一定阈值的标注位置输出成对

img = cv2.imread('C:/Users/lurchy/Desktop/EasyDL/8230-4502-0.5.jpg')
cross = crossing['crossing']
#读取图片

spot_add = []
qian = []
m=0
for i in cross:
    qian.append(int((i[m]+i[m+2])/2))
    qian.append(int((i[m+1]+i[m+3])/2))
    spot_add.append(qian)
    qian = []
#计算像素点距离

def cv_imshow(img,name):
    cv2.imshow(name,img)
    cv2.waitKey()
    cv2.destroyAllWindows()
#图片显示函数
grad_ = []
q=0
w=0
for i in range(len(spot_add)):
    for s in range(len(spot_add)):
        if((i+s)>len(spot_add)-1):
            continue
        crop_img = img[spot_add[i][1]:spot_add[i+s][1],spot_add[i][0]:spot_add[i+s][0]]
        if(crop_img.size>0):
#             cv_imshow(crop_img,'a')
            sobely = cv2.Sobel(crop_img,cv2.CV_64F,0,1,ksize=3)
            sobely = cv2.convertScaleAbs(sobely)
            for e in sobely:
                for j in e:
                    for k in j:
                        if k!=0:
                            w+=1
                            q+=k
            grad = q/w#
            grad_.append(grad)
# grad_.sort()
# #             cv_imshow(sobely,'a')
#             #设置梯度阈值
#     #输出符合阈值的坐标对
# #求标注区域对之间的梯度，并设置阈值，输出符合要求的坐标对

# pass
#数闭环个数（小闭环）
    

In [4]:
s=0
for i in grad_:
    s+=i
print(s/(len(grad_)))

41.80993804824114


In [ ]:
# grad_=grad_.sort()
# s=0
# for i in grad_:
#     if(i==grad_[0]or i==grad_[len(grad_)-1]):
#         continue
#     else:
#         s+=i
# s/(len(grad_)-2)

In [ ]:
crop_ = img[279:318,251:262]
cv_imshow(crop_,'a')
sobely = cv2.Sobel(crop_,cv2.CV_64F,0,1,ksize=3)
sobely = cv2.convertScaleAbs(sobely)
s=0
m=0
for i in sobely:
    for j in i:
        for k in j:
            if k!=0:
                m+=1
                s+=k
grad = s/m#
cv_imshow(sobely,'a')
sobely

In [ ]:
spot_add = np.array(spot_add)
points = spot_add
points[:,[0,1]]=points[:,[1,0]]
# vor  = Voronoi(points=points)
tri = Delaunay(points)
# voronoi_plot_2d(vor)
plt.triplot(points[:,0], points[:,1], tri.simplices.copy())
plt.plot(points[:,0],points[:,1],'o')
plt.show()#Delaunay算法画边界

In [ ]:
#退化，计算需要去掉的边
edge = points[tri.simplices]#三角形点集合
del_b = []#去边列表
def grad_(m,n):#输入两个坐标，计算坐标之间梯度
    crop_ = img[m[1]:n[1],m[0],n[0]]
    sobely = cv2.Sobel(crop_img,cv2.CV_64F,0,1,ksize=3)
    sobely = cv2.convertScaleAbs(sobely)
    s=0
    for i in sobely:
        for j in i:
            for k in j:
                s+=k
    grad = s/2#梯度转化为与阈值判断的标准数据格式
    return grad
def ggb_c(m,n):#公共边选取
    ggb = []
    for i in m:
        for j in n:
            p = 0
            if(set(i)==set(j)):
                ggb.append(list(i))#公共边
    if(len(ggb)==2):
        if(grad_(m,n)>):#计算梯度阈值
            k =1 #需要加入置1
        else:
            k=0
#     print(ggb)
#     print(len(ggb))
#     print('***')
    ggb=[]#释放
#     return k
for i in range(len(edge)):
    for j in range(len(edge)):
        if(i!=j):
#             compare_(edge[i],edge[j])
            k = compare_(edge[i],edge[j])
            if(k==1):
                gd=[]
                gd.append(edge[i],edge[j])
                del_b.append(gd)#将该坐标对加入到去边列表中

In [ ]:
#画图，有一个坐标数组，默认里面都相连，有一个去边数组，里面是不需要连接的坐标对，怎样可视化出来
#使用白线覆盖掉
spot_add = np.array(spot_add)
points = spot_add
points[:,[0,1]]=points[:,[1,0]]
# vor  = Voronoi(points=points)
tri = Delaunay(points)
# voronoi_plot_2d(vor)
plt.triplot(points[:,0], points[:,1], tri.simplices.copy())
plt.plot(points[:,0],points[:,1],'o')
for i in range(len(del_b)):
    for j in range(0,2):
        plt.plot(del_b[i][j][0],del_b[i][j][1],color = '#FFFFFF')

In [ ]:
edge[3]

In [ ]:
edge

In [ ]:
crop_ = img[279:318,251:262]
cv_imshow(crop_,'a')
sobely = cv2.Sobel(crop_,cv2.CV_64F,0,1,ksize=3)
sobely = cv2.convertScaleAbs(sobely)
s=0
m=0
for i in sobely:
    for j in i:
        for k in j:
            if k!=0:
                m+=1
                s+=k
grad = s/m#
cv_imshow(sobely,'a')
sobely

In [ ]:
grad

In [ ]:
for i in sobely:
    print(i)

In [ ]:
a=[1,2]
b=[3,4]
c=[]
c.append(a)
c.append(b)
d=[]
d.append(c)
e = [5,6]
f = [6,7]
m=[]
m.append(e)
m.append(f)
d.append(m)

In [ ]:
grad

In [ ]:
plt.plot(spot_add[:,1],spot_add[:,0])
plt.show()